# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [ ]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
!unzip -q food-11.zip

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
From (redirected): https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy&confirm=t&uuid=f1e8d07d-b929-487b-8b64-eb27be36a98c
To: /content/food-11.zip
100% 963M/963M [00:09<00:00, 96.5MB/s]
replace food-11/training/unlabeled/00/5176.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [ ]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# This is for the progress bar.
from tqdm.auto import tqdm

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [ ]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.02,
    ),
    transforms.ToTensor(),
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [ ]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),

            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),   # 全連接層前加 dropout 抗 overfit

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(256, 11),
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [1]:
from torch.utils.data import Dataset

class PseudoLabelDataset(Dataset):
    """
    用來包住：某個 base_dataset + 我們挑出來的 indices + 對應的 pseudo labels
    之後可以直接丟進 DataLoader 再跟 train_set 用 ConcatDataset 合併。
    """
    def __init__(self, base_dataset, indices, pseudo_labels):
        self.base_dataset = base_dataset      # 通常就是 unlabeled_set
        self.indices = indices                # list[int]
        self.pseudo_labels = pseudo_labels    # list[int]，和 indices 對應

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        # 取出在 base_dataset 中對應的那一張圖片
        real_idx = self.indices[idx]
        img, _ = self.base_dataset[real_idx]  # 忽略原本的假 label
        label = self.pseudo_labels[idx]
        return img, label


def get_pseudo_labels(dataset, model, threshold=0.95, max_ratio=0.4):
    """
    用目前的 model 幫 dataset 產生 pseudo labels：
    - 先對整個 unlabeled dataset 做推論，記下每一張的 conf & pred
    - 只留下 max prob >= threshold 的樣本
    - 如果留下來太多，再依照 conf 排序，只取前 max_ratio 比例
    - 回傳一個新的 Dataset（PseudoLabelDataset）
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 用固定順序掃過整個 dataset（shuffle 一定要 False）
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    model.eval()
    softmax = nn.Softmax(dim=-1)

    all_confs = []
    all_preds = []
    all_indices = []

    offset = 0
    for imgs, _ in tqdm(data_loader):
        imgs = imgs.to(device)

        with torch.no_grad():
            logits = model(imgs)          # [B, 11]
            probs = softmax(logits)       # [B, 11]

        conf, preds = torch.max(probs, dim=1)   # [B], [B]

        bsz = imgs.size(0)
        for i in range(bsz):
            all_confs.append(conf[i].item())
            all_preds.append(preds[i].cpu().item())
            all_indices.append(offset + i)

        offset += bsz

    all_confs = torch.tensor(all_confs)
    all_preds = torch.tensor(all_preds, dtype=torch.long)
    all_indices = torch.tensor(all_indices, dtype=torch.long)

    # 先做 threshold 篩選
    mask = all_confs >= threshold
    cand_confs = all_confs[mask]
    cand_preds = all_preds[mask]
    cand_indices = all_indices[mask]

    total_unlabeled = len(dataset)
    max_selected = int(total_unlabeled * max_ratio)

    if cand_indices.size(0) > max_selected and max_selected > 0:
        # 太多的話，依照 conf 由大到小排序，只取前 max_selected 筆
        sorted_conf, sort_idx = torch.sort(cand_confs, descending=True)
        sort_idx = sort_idx[:max_selected]

        selected_indices = cand_indices[sort_idx].tolist()
        selected_labels = cand_preds[sort_idx].tolist()
    else:
        selected_indices = cand_indices.tolist()
        selected_labels = cand_preds.tolist()

    model.train()

    print(f"[Pseudo-Label] selected {len(selected_indices)} / {len(dataset)} samples "
          f"(threshold={threshold}, max_ratio={max_ratio})")

    pseudo_dataset = PseudoLabelDataset(dataset, selected_indices, selected_labels)
    return pseudo_dataset

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# The number of training epochs.
n_epochs = 80

# ----- 半監督開關 & 策略設定 -----
do_semi = True           # 想用半監督就設 True
semi_start_epoch = 20    # 從第 20 個 epoch 開始 pseudo-label（比之前早的 40 還要小心地用高 threshold）

def get_pl_threshold(epoch_idx):
    """
    根據 epoch 動態調整 pseudo-label threshold
    epoch_idx 是從 0 開始的，所以要 +1 才是人類看到的 epoch
    """
    e = epoch_idx + 1
    if e < 40:
        return 0.95      # 初期非常保守，只用超高信心的 pseudo-label
    else:
        return 0.90      # 後期可以稍微放寬一點


best_valid_acc = 0.0
best_model_path = "best_model.ckpt"

for epoch in range(n_epochs):
    # ---------- Semi-supervised: 每個 epoch 重新 pseudo-label ----------
    if do_semi and (epoch + 1) >= semi_start_epoch:
        thr = get_pl_threshold(epoch)
        print(f"[ Semi-supervised ] Relabel unlabeled data at epoch {epoch+1} (threshold={thr})")
        pseudo_set = get_pseudo_labels(
            unlabeled_set,
            model,
            threshold=thr,
            max_ratio=0.4,   # 一次最多只用 40% 的 unlabeled
        )

        if len(pseudo_set) > 0:
            concat_dataset = ConcatDataset([train_set, pseudo_set])
        else:
            # 若這個 threshold 太嚴格，完全沒選到，就只用原本 labeled data
            concat_dataset = train_set

        train_loader = DataLoader(
            concat_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True,
        )
    # 如果還沒到 semi_start_epoch，就使用一開始建好的 train_loader（只含 labeled）

    # ---------- Training ----------
    model.train()

    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        imgs, labels = batch
        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))

        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        train_loss.append(loss.item())
        train_accs.append(acc)

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    model.eval()

    valid_loss = []
    valid_accs = []

    for batch in tqdm(valid_loader):
        imgs, labels = batch
        with torch.no_grad():
            logits = model(imgs.to(device))
            loss = criterion(logits, labels.to(device))

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"=> Save best model (epoch {epoch+1}, valid_acc = {best_valid_acc:.5f})")

  0%|          | 0/57 [00:00<?, ?it/s]

[ Train | 001/080 ] loss = 2.06224, acc = 0.31422


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 001/080 ] loss = 2.49009, acc = 0.17604
=> Save best model (epoch 1, valid_acc = 0.17604)


  0%|          | 0/57 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 002/080 ] loss = 2.25769, acc = 0.26693
=> Save best model (epoch 2, valid_acc = 0.26693)


  0%|          | 0/57 [00:00<?, ?it/s]

[ Train | 003/080 ] loss = 1.59743, acc = 0.46754


  0%|          | 0/6 [00:00<?, ?it/s]

## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [ ]:
# 檢查 best_model 在 validation set 上的表現是不是跟當初 log 一樣
model = Classifier().to(device)
model.load_state_dict(torch.load("best_model.ckpt"))
model.eval()

val_accs = []
val_losses = []

for batch in valid_loader:
    imgs, labels = batch
    with torch.no_grad():
        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
    acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
    val_accs.append(acc)
    val_losses.append(loss.item())

print("Loaded best_model - valid loss:", sum(val_losses)/len(val_losses),
      "valid acc:", sum(val_accs)/len(val_accs))

Loaded best_model - valid loss: 1.4403901894887288 valid acc: tensor(0.5880, device='cuda:0')


In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")